# Final Exam for Financial Economics III (NYU)
Prof. Sydney Ludvigson


Steven Zheng
<br>steven.zheng@stern.nyu.edu
<br>May 2019


This notebook replicates Table IV of Lettau, Ludvigson, and Ma (2019). Specifically, it runs Fama-MacBeth regressions with the capital share model, Fama-French 3-factor model and a horse race between the two. Secion 3 uses 25 size/book-to-market sorted portfolios as the test assets and section 4 uses all equity portfolios pooled together. Data are from the Journal of Finance's website.

# 1. Set up

#### Directories
These directories contain the clean data and functions, respectively. In directory_rawfunctions:
- functions_data.py contains functions to load and transform the data;
- functions_famamacbeth.py contains functions necessary to run the Fama-MacBeth regressions.

In [1]:
directory_cleandata = '../CleanData/'
directory_functions = '../Functions/'

#### Import settings and functions

In [2]:
from Global.import_libraries import *
from Global.global_variables_y import *
from Functions.functions_data import *
from Functions.functions_famamacbeth import *

# 2. Load data

In [3]:
data_all = load_data(directory_data=directory_cleandata,
                     file_name='Replication_Data.xlsx')

Here's a snippet of what the data looks like. Naming convention follow Fama and French. H[#]Q denotes the horizon in quarters ahead:
- H0Q is contemporaneous (i.e., for the CS regressions)
- H4Q and H8Q are 4- and 8-quarter ahead for the TS regressions.

In [4]:
data_all.head()

,Consumption,Labor Share,Rm H0Q,SMB H0Q,HML H0Q,MoM,LevFac,KS H0Q,KS H4Q,KS H8Q,SMALL LoBM H0Q,ME1 BM2 H0Q,ME1 BM3 H0Q,ME1 BM4 H0Q,SMALL HiBM H0Q,ME2 BM1 H0Q,ME2 BM2 H0Q,ME2 BM3 H0Q,ME2 BM4 H0Q,ME2 BM5 H0Q,ME3 BM1 H0Q,ME3 BM2 H0Q,ME3 BM3 H0Q,ME3 BM4 H0Q,ME3 BM5 H0Q,ME4 BM1 H0Q,ME4 BM2 H0Q,ME4 BM3 H0Q,ME4 BM4 H0Q,ME4 BM5 H0Q,BIG LoBM H0Q,ME5 BM2 H0Q,ME5 BM3 H0Q,ME5 BM4 H0Q,BIG HiBM H0Q,SMALL LoBM H4Q,ME1 BM2 H4Q,ME1 BM3 H4Q,ME1 BM4 H4Q,SMALL HiBM H4Q,ME2 BM1 H4Q,ME2 BM2 H4Q,ME2 BM3 H4Q,ME2 BM4 H4Q,ME2 BM5 H4Q,ME3 BM1 H4Q,ME3 BM2 H4Q,ME3 BM3 H4Q,ME3 BM4 H4Q,ME3 BM5 H4Q,ME4 BM1 H4Q,ME4 BM2 H4Q,ME4 BM3 H4Q,ME4 BM4 H4Q,ME4 BM5 H4Q,BIG LoBM H4Q,ME5 BM2 H4Q,ME5 BM3 H4Q,ME5 BM4 H4Q,BIG HiBM H4Q,SMALL LoBM H8Q,ME1 BM2 H8Q,ME1 BM3 H8Q,ME1 BM4 H8Q,SMALL HiBM H8Q,ME2 BM1 H8Q,ME2 BM2 H8Q,ME2 BM3 H8Q,ME2 BM4 H8Q,ME2 BM5 H8Q,ME3 BM1 H8Q,ME3 BM2 H8Q,ME3 BM3 H8Q,ME3 BM4 H8Q,ME3 BM5 H8Q,ME4 BM1 H8Q,ME4 BM2 H8Q,ME4 BM3 H8Q,ME4 BM4 H8Q,ME4 BM5 H8Q,BIG LoBM H8Q,ME5 BM2 H8Q,ME5 BM3 H8Q,ME5 BM4 H8Q,BIG HiBM H8Q,SMALL LoOP H0Q,ME1 OP2 H0Q,ME1 OP3 H0Q,ME1 OP4 H0Q,SMALL HiOP H0Q,ME2 OP1 H0Q,ME2 OP2 H0Q,ME2 OP3 H0Q,ME2 OP4 H0Q,ME2 OP5 H0Q,ME3 OP1 H0Q,ME3 OP2 H0Q,ME3 OP3 H0Q,ME3 OP4 H0Q,ME3 OP5 H0Q,ME4 OP1 H0Q,ME4 OP2 H0Q,ME4 OP3 H0Q,ME4 OP4 H0Q,ME4 OP5 H0Q,BIG LoOP H0Q,ME5 OP2 H0Q,ME5 OP3 H0Q,ME5 OP4 H0Q,BIG HiOP H0Q,SMALL LoOP H4Q,ME1 OP2 H4Q,ME1 OP3 H4Q,ME1 OP4 H4Q,SMALL HiOP H4Q,ME2 OP1 H4Q,ME2 OP2 H4Q,ME2 OP3 H4Q,ME2 OP4 H4Q,ME2 OP5 H4Q,ME3 OP1 H4Q,ME3 OP2 H4Q,ME3 OP3 H4Q,ME3 OP4 H4Q,ME3 OP5 H4Q,ME4 OP1 H4Q,ME4 OP2 H4Q,ME4 OP3 H4Q,ME4 OP4 H4Q,ME4 OP5 H4Q,BIG LoOP H4Q,ME5 OP2 H4Q,ME5 OP3 H4Q,ME5 OP4 H4Q,BIG HiOP H4Q,SMALL LoOP H8Q,ME1 OP2 H8Q,ME1 OP3 H8Q,ME1 OP4 H8Q,SMALL HiOP H8Q,ME2 OP1 H8Q,ME2 OP2 H8Q,ME2 OP3 H8Q,ME2 OP4 H8Q,ME2 OP5 H8Q,ME3 OP1 H8Q,ME3 OP2 H8Q,ME3 OP3 H8Q,ME3 OP4 H8Q,ME3 OP5 H8Q,ME4 OP1 H8Q,ME4 OP2 H8Q,ME4 OP3 H8Q,ME4 OP4 H8Q,ME4 OP5 H8Q,BIG LoOP H8Q,ME5 OP2 H8Q,ME5 OP3 H8Q,ME5 OP4 H8Q,BIG HiOP H8Q,SMALL LoINV H0Q,ME1 INV2 H0Q,ME1 INV3 H0Q,ME1 INV4 H0Q,SMALL HiINV H0Q,ME2 INV1 H0Q,ME2 INV2 H0Q,ME2 INV3 H0Q,ME2 INV4 H0Q,ME2 INV5 H0Q,ME3 INV1 H0Q,ME3 INV2 H0Q,ME3 INV3 H0Q,ME3 INV4 H0Q,ME3 INV5 H0Q,ME4 INV1 H0Q,ME4 INV2 H0Q,ME4 INV3 H0Q,ME4 INV4 H0Q,ME4 INV5 H0Q,BIG LoINV H0Q,ME5 INV2 H0Q,ME5 INV3 H0Q,ME5 INV4 H0Q,BIG HiINV H0Q,SMALL LoINV H4Q,ME1 INV2 H4Q,ME1 INV3 H4Q,ME1 INV4 H4Q,SMALL HiINV H4Q,ME2 INV1 H4Q,ME2 INV2 H4Q,ME2 INV3 H4Q,ME2 INV4 H4Q,ME2 INV5 H4Q,ME3 INV1 H4Q,ME3 INV2 H4Q,ME3 INV3 H4Q,ME3 INV4 H4Q,ME3 INV5 H4Q,ME4 INV1 H4Q,ME4 INV2 H4Q,ME4 INV3 H4Q,ME4 INV4 H4Q,ME4 INV5 H4Q,BIG LoINV H4Q,ME5 INV2 H4Q,ME5 INV3 H4Q,ME5 INV4 H4Q,BIG HiINV H4Q,SMALL LoINV H8Q,ME1 INV2 H8Q,ME1 INV3 H8Q,ME1 INV4 H8Q,SMALL HiINV H8Q,ME2 INV1 H8Q,ME2 INV2 H8Q,ME2 INV3 H8Q,ME2 INV4 H8Q,ME2 INV5 H8Q,ME3 INV1 H8Q,ME3 INV2 H8Q,ME3 INV3 H8Q,ME3 INV4 H8Q,ME3 INV5 H8Q,ME4 INV1 H8Q,ME4 INV2 H8Q,ME4 INV3 H8Q,ME4 INV4 H8Q,ME4 INV5 H8Q,BIG LoINV H8Q,ME5 INV2 H8Q,ME5 INV3 H8Q,ME5 INV4 H8Q,BIG HiINV H8Q,Lo PRIOR H0Q,PRIOR 2 H0Q,PRIOR 3 H0Q,PRIOR 4 H0Q,PRIOR 5 H0Q,PRIOR 6 H0Q,PRIOR 7 H0Q,PRIOR 8 H0Q,PRIOR 9 H0Q,Hi PRIOR H0Q,Lo PRIOR H4Q,PRIOR 2 H4Q,PRIOR 3 H4Q,PRIOR 4 H4Q,PRIOR 5 H4Q,PRIOR 6 H4Q,PRIOR 7 H4Q,PRIOR 8 H4Q,PRIOR 9 H4Q,Hi PRIOR H4Q,Lo PRIOR H8Q,PRIOR 2 H8Q,PRIOR 3 H8Q,PRIOR 4 H8Q,PRIOR 5 H8Q,PRIOR 6 H8Q,PRIOR 7 H8Q,PRIOR 8 H8Q,PRIOR 9 H8Q,Hi PRIOR H8Q
1963-12-31,9.428028,63.431566,1.0355,0.9624,1.0136,4.106680,1.562966,0.365684,0.995839,1.052133,-0.055459,-0.063120,-0.021427,-0.035502,-0.006146,-0.045654,0.006656,0.019157,0.022927,0.030740,-0.009966,-0.007969,0.024460,0.019131,-0.005500,-0.021822,0.004665,0.025710,0.090308,0.097119,0.058614,0.042234,-0.004334,0.017816,0.034043,0.104407,0.106222,0.163527,0.102193,0.228979,0.039824,0.018307,0.224291,0.118273,0.249387,0.016973,0.149139,0.174242,0.217849,0.200113,0.063026,0.185578,0.176579,0.296012,0.169143,0.135473,0.085549,0.159756,0.156505,0.129980,0.379854,0.572005,0.465723,0.528504,0.801165,0.362679,0.305496,0.570982,0.508687,0.687609,0.291532,0.474880,0.559047,0.558881,0.692656,0.282378,0.4256

# 3. Fama-MacBeth
This section runs the Fama-MacBeth regressions. To run, specify:
1. data_all: panda containing the data
2. variables_y: the test assets
3. variables_X: the SDF
4. horizon: the horizon (in quarters) to measure factor exposures.

See functions_famamacbeth.py for the full return structure, but ['results'] will return the point estimates, Fama-MacBeth and Shanken-corrected t-stats, R-squared, RMSE/RMSR and BIC.

## 3.1 Size/BM

#### Capital share only

In [5]:
variables_y = me_beme
variables_X = ['KS']
horizon = 8
mebeme_ks = fama_macbeth(data_all=data_all,
                         variables_y=variables_y,
                         variables_X=variables_X,
                         horizon=horizon)

In [6]:
mebeme_ks['results']

,Const,KS,R-squared,RMSE/RMSR,BIC
Mean,1.55,0.67,0.8,0.12,-283.41
t-stats Fama-MacBeth,2.19,4.47,,,
t-stats Shanken,2.15,4.39,,,


#### FF 3-factor only

In [7]:
variables_y = me_beme
variables_X = ['Rm','SMB','HML']
horizon = 0
mebeme_ff = fama_macbeth(data_all=data_all,
                         variables_y=variables_y,
                         variables_X=variables_X,
                         horizon=horizon)

In [8]:
mebeme_ff['results']

,Const,Rm,SMB,HML,R-squared,RMSE/RMSR,BIC
Mean,3.63,-1.96,0.70,1.35,0.69,0.15,-268.12
t-stats Fama-MacBeth,3.79,-1.72,1.67,2.64,,,
t-stats Shanken,3.54,-1.61,1.56,2.47,,,


#### Horse race: KS vs. FF 3-factor

In [9]:
variables_y = me_beme
variables_X1 = ['KS']
variables_X2 = ['Rm','SMB','HML']
horizon1 = 8
horizon2 = 0
mebeme_ff = fama_macbeth_horserace(data_all=data_all,
                                   variables_y=variables_y,
                                   variables_X1=variables_X1,
                                   variables_X2=variables_X2,
                                   horizon1=horizon1,
                                   horizon2=horizon2)

In [10]:
mebeme_ff['results']

,Const,KS,Rm,SMB,HML,R-squared,RMSE/RMSR,BIC
Mean,3.57,0.50,-2.04,0.23,0.23,0.84,0.1,-282.29
t-stats Fama-MacBeth,3.73,3.98,-1.79,0.52,0.42,,,
t-stats Shanken,3.43,3.66,-1.65,0.47,0.39,,,


## 3.2 All equities

#### Capital share only

In [11]:
variables_y = me_beme + prior + me_inv + me_op
variables_X = ['KS']
horizon = 8
all_ks = fama_macbeth(data_all=data_all,
                      variables_y=variables_y,
                      variables_X=variables_X,
                      horizon=horizon)

In [12]:
all_ks['results']

,Const,KS,R-squared,RMSE/RMSR,BIC
Mean,1.65,0.57,0.74,0.14,-966.12
t-stats Fama-MacBeth,2.50,3.88,,,
t-stats Shanken,2.47,3.82,,,


#### FF 3-factor only

In [13]:
variables_y = me_beme + prior + me_inv + me_op
variables_X = ['Rm','SMB','HML']
horizon = 0
all_ff = fama_macbeth(data_all=data_all,
                      variables_y=variables_y,
                      variables_X=variables_X,
                      horizon=horizon)

In [14]:
all_ff['results']

,Const,Rm,SMB,HML,R-squared,RMSE/RMSR,BIC
Mean,3.02,-1.28,0.67,1.37,0.68,0.15,-943.11
t-stats Fama-MacBeth,3.31,-1.19,1.61,2.68,,,
t-stats Shanken,3.16,-1.13,1.54,2.56,,,


#### Horse race: KS vs. FF 3-factor

In [15]:
variables_y = me_beme + prior + me_inv + me_op
variables_X1 = ['KS']
variables_X2 = ['Rm','SMB','HML']
horizon1 = 8
horizon2 = 0
all_ff = fama_macbeth_horserace(data_all=data_all,
                                   variables_y=variables_y,
                                   variables_X1=variables_X1,
                                   variables_X2=variables_X2,
                                   horizon1=horizon1,
                                   horizon2=horizon2)

In [16]:
all_ff['results']

,Const,KS,Rm,SMB,HML,R-squared,RMSE/RMSR,BIC
Mean,2.89,0.39,-1.25,0.25,0.40,0.78,0.12,-970.29
t-stats Fama-MacBeth,3.20,3.44,-1.16,0.56,0.70,,,
t-stats Shanken,3.07,3.30,-1.11,0.53,0.67,,,
